In [13]:
import pandas as pd
from statistics import median
import csv
from projects import project_list

In [14]:
r_file_name = 'new_dynamic_batching.csv'
result_file = open(r_file_name, 'w')
result_headers = ['project', 'algorithm', 'batch_size', 'builds_saved', 'median_delay', 'testall_size']
writer = csv.writer(result_file)
writer.writerow(result_headers)

69

In [15]:
batch_total = 0

In [16]:
def output_values(tr_results):
    l = []
    i = 0
    while i < len(tr_results):
        if tr_results[i] == 'passed':
            l.append(1)
        else:
            l.append(0)
        i += 1
    
    return l

In [17]:
def batchstop4(commits):
    global batch_total
    
    l = len(commits)//2
    
    batch_total += 1        
    
    if 0 in commits:
        
        if len(commits) == 4:
            batch_total += 4
            return commits[0] & commits[1] & commits[2] & commits[3]
        
        return batchbisect(commits[:l]) & batchbisect(commits[l:])
    
    return 1

In [18]:
def batchbisect(commits):
    global batch_total
    
    l = len(commits)//2
    
    batch_total += 1
    if len(commits) == 1:
        return commits[0]
    
    if 0 in commits:
        return batchbisect(commits[:l]) & batchbisect(commits[l:])
    
    return 1

In [19]:
def batchbisect_dynamic(commits):
    global batch_total
    
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    
    while i < length:
        batch = commits[i:i+batch_size]
        fails_per = 100*batch.count(0)/batch_size
        
        delays.extend([batch_size-x-1 for x in range(batch_size)])
        
        batch_total = 0
        res = batchbisect(batch)
        builds += batch_total
        print('Building {} {} {} {}'.format(batch_size, batch, batch_total, builds))
        
        i += batch_size
        
        if res == 0:
            
            if fails_per > 50 & batch[-1] == 0:
                #if batch has a lot of failures then definitely split
                
                if batch_size == 1:
                    continue
                else:
                    batch_size //= 2
            
            if fails_per < 20 & batch[-1] == 1:
                continue
        
        else:
            if batch_size == 16:
                continue
            else:
                batch_size *= 2
    
    return builds, delays
            

In [20]:
def batchstop4_dynamic(commits):
    global batch_total
    
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    
    while i < length:
        batch = commits[i:i+batch_size]
        fails_per = 100*batch.count(0)/batch_size
        delays.extend([batch_size-x-1 for x in range(batch_size)])
        
        batch_total = 0
        res = batchstop4(batch)
        builds += batch_total
        print('Building {} {} {} {}'.format(batch_size, batch, batch_total, builds))
        
        i += batch_size
        
        if res == 0:
            if fails_per > 50 & batch[-1] == 0:
                #if batch has a lot of failures then definitely split
                
                if batch_size == 4:
                    continue
                else:
                    batch_size //= 2
            
            if fails_per < 20 & batch[-1] == 1:
                continue
        
        else:
            if batch_size == 16:
                continue
            else:
                batch_size *= 2
    
    return builds, delays

In [21]:
discard_list = []
for p in project_list:
    data = pd.read_csv('data/25_1_travis_data/'+p)
    branch_type = data['git_branch'].tolist()
    if 'master' in branch_type:
        data = data[ data['git_branch'] == 'master']
    else:
        data = data[ data['git_branch'] == 'trunk']
        
    data = data.iloc[100:]
    if len(data) == 0:
        print('Could not continue for {}'.format(p))
        discard_list.append(p)
        continue
        
    results = output_values(data['tr_status'].tolist())
    builds, delays = batchbisect_dynamic(results)
    reqd = 100-(100*builds/len(results))
    fr = 100*results.count(0)/len(results)
    
    p_name = p.split('/')[1]
    writer.writerow([p_name, 'BATCHBISECT', '-1', reqd, median(delays), len(results)])
    
    

Could not continue for BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv
Building 16 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1] 15 15
Building 8 [0, 1, 1, 1, 1, 1, 1, 1] 7 22
Building 4 [1, 1, 1, 1] 1 23
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 24
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 33
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 34
Building 16 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1] 19 53
Building 8 [1, 1, 1, 1, 0, 1, 1, 1] 7 60
Building 4 [0, 1, 1, 1] 5 65
Building 2 [0, 1] 3 68
Building 1 [1] 1 69
Building 2 [0, 1] 3 72
Building 1 [1] 1 73
Building 2 [0, 1] 3 76
Building 1 [1] 1 77
Building 2 [0, 1] 3 80
Building 1 [1] 1 81
Building 2 [1, 1] 1 82
Building 4 [1, 1, 1, 1] 1 83
Building 8 [1, 0, 1, 1, 1, 1, 1, 1] 7 90
Building 4 [1, 1, 1, 1] 1 91
Building 8 [1, 1, 1, 1, 0, 1, 0, 1] 9 100
Building 4 [1, 1, 1, 1] 1 101
Building 8 [0, 0, 1, 0, 1, 1, 0, 0] 13 114
Building 4 [0, 0, 1, 0] 7 121
Building 2 [1, 0] 3 124
Building 1 [0] 1 125
Building 1 [1] 1 126
Bu

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1] 11 11
Building 8 [1, 1, 1, 1, 1, 1, 1, 0] 7 18
Building 4 [0, 1, 1, 1] 5 23
Building 2 [1, 1] 1 24
Building 4 [1, 1, 1, 1] 1 25
Building 8 [1, 1, 1, 0, 0, 1, 1, 1] 11 36
Building 4 [1, 1, 1, 1] 1 37
Building 8 [1, 0, 0, 0, 0, 0, 0, 0] 15 52
Building 4 [0, 0, 0, 1] 7 59
Building 2 [1, 1] 1 60
Building 4 [1, 1, 1, 1] 1 61
Building 8 [1, 1, 1, 1, 1, 1, 0, 1] 7 68
Building 4 [1, 1, 1, 1] 1 69
Building 8 [1, 1, 1, 1, 1, 1, 1, 0] 7 76
Building 4 [1, 1, 1, 1] 1 77
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 78
Building 16 [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1] 21 99
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 100
Building 16 [1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1] 23 123
Building 8 [1, 1, 1, 0, 1, 1, 1, 1] 7 130
Building 4 [1, 1, 1, 1] 1 131
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 132
Building 16 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1] 15 147
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 148
Building 16 [0, 1, 1, 1, 1, 1, 0, 1

Building 4 [1, 1, 1, 1] 1 423
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 424
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 425
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] 11 436
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 437
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 438
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 439
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] 11 450
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 451
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 452
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] 11 463
Building 8 [1, 1, 1, 1, 0, 0, 0, 0] 9 472
Building 4 [1, 1, 1, 1] 1 473
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 474
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] 11 485
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 486
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 487
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 488
Building 16 [1

Building 1 [0] 1 789
Building 1 [0] 1 790
Building 1 [0] 1 791
Building 1 [0] 1 792
Building 1 [0] 1 793
Building 1 [0] 1 794
Building 1 [0] 1 795
Building 1 [0] 1 796
Building 1 [0] 1 797
Building 1 [0] 1 798
Building 1 [0] 1 799
Building 1 [0] 1 800
Building 1 [0] 1 801
Building 1 [0] 1 802
Building 1 [0] 1 803
Building 1 [0] 1 804
Building 1 [0] 1 805
Building 1 [0] 1 806
Building 1 [0] 1 807
Building 1 [0] 1 808
Building 1 [0] 1 809
Building 1 [0] 1 810
Building 1 [0] 1 811
Building 1 [0] 1 812
Building 1 [0] 1 813
Building 1 [0] 1 814
Building 1 [0] 1 815
Building 1 [0] 1 816
Building 1 [0] 1 817
Building 1 [0] 1 818
Building 1 [0] 1 819
Building 1 [0] 1 820
Building 1 [0] 1 821
Building 1 [0] 1 822
Building 1 [0] 1 823
Building 1 [0] 1 824
Building 1 [0] 1 825
Building 1 [0] 1 826
Building 1 [0] 1 827
Building 1 [0] 1 828
Building 1 [0] 1 829
Building 1 [0] 1 830
Building 1 [0] 1 831
Building 1 [0] 1 832
Building 1 [0] 1 833
Building 1 [0] 1 834
Building 1 [0] 1 835
Building 1 [0

Building 1 [0] 1 1792
Building 1 [0] 1 1793
Building 1 [0] 1 1794
Building 1 [0] 1 1795
Building 1 [0] 1 1796
Building 1 [0] 1 1797
Building 1 [0] 1 1798
Building 1 [0] 1 1799
Building 1 [0] 1 1800
Building 1 [0] 1 1801
Building 1 [0] 1 1802
Building 1 [0] 1 1803
Building 1 [0] 1 1804
Building 1 [0] 1 1805
Building 1 [0] 1 1806
Building 1 [0] 1 1807
Building 1 [0] 1 1808
Building 1 [0] 1 1809
Building 1 [0] 1 1810
Building 1 [0] 1 1811
Building 1 [0] 1 1812
Building 1 [0] 1 1813
Building 1 [0] 1 1814
Building 1 [0] 1 1815
Building 1 [0] 1 1816
Building 1 [0] 1 1817
Building 1 [0] 1 1818
Building 1 [0] 1 1819
Building 1 [0] 1 1820
Building 1 [0] 1 1821
Building 1 [0] 1 1822
Building 1 [0] 1 1823
Building 1 [0] 1 1824
Building 1 [0] 1 1825
Building 1 [0] 1 1826
Building 1 [0] 1 1827
Building 1 [0] 1 1828
Building 1 [0] 1 1829
Building 1 [0] 1 1830
Building 1 [0] 1 1831
Building 1 [0] 1 1832
Building 1 [0] 1 1833
Building 1 [0] 1 1834
Building 1 [0] 1 1835
Building 1 [0] 1 1836
Building 1

...

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1] 11 11
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 12
Building 16 [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 23
Building 8 [1, 1, 1, 1, 1, 1, 1, 0] 7 30
Building 4 [0, 1, 1, 1] 5 35
Building 2 [1, 0] 3 38
Building 1 [0] 1 39
Building 1 [1] 1 40
Building 2 [1, 1] 1 41
Building 4 [1, 1, 1, 0] 5 46
Building 2 [0, 1] 3 49
Building 1 [1] 1 50
Building 2 [1, 1] 1 51
Building 4 [1, 1, 1, 1] 1 52
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 53
Building 16 [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 64
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 65
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 66
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 67
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 68
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 69
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 70
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1] 13 83

Building 1 [0] 1 1730
Building 1 [0] 1 1731
Building 1 [0] 1 1732
Building 1 [0] 1 1733
Building 1 [0] 1 1734
Building 1 [0] 1 1735
Building 1 [0] 1 1736
Building 1 [0] 1 1737
Building 1 [0] 1 1738
Building 1 [0] 1 1739
Building 1 [0] 1 1740
Building 1 [0] 1 1741
Building 1 [0] 1 1742
Building 1 [0] 1 1743
Building 1 [0] 1 1744
Building 1 [0] 1 1745
Building 1 [0] 1 1746
Building 1 [0] 1 1747
Building 1 [0] 1 1748
Building 1 [0] 1 1749
Building 1 [0] 1 1750
Building 1 [0] 1 1751
Building 1 [0] 1 1752
Building 1 [0] 1 1753
Building 1 [0] 1 1754
Building 1 [0] 1 1755
Building 1 [0] 1 1756
Building 1 [0] 1 1757
Building 1 [0] 1 1758
Building 1 [0] 1 1759
Building 1 [0] 1 1760
Building 1 [0] 1 1761
Building 1 [0] 1 1762
Building 1 [0] 1 1763
Building 1 [0] 1 1764
Building 1 [0] 1 1765
Building 1 [0] 1 1766
Building 1 [0] 1 1767
Building 1 [0] 1 1768
Building 1 [0] 1 1769
Building 1 [0] 1 1770
Building 1 [0] 1 1771
Building 1 [0] 1 1772
Building 1 [0] 1 1773
Building 1 [0] 1 1774
Building 1

Building 16 [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 13 13
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 14
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 15
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 16
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 17
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 18
Building 16 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1] 15 33
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 34
Building 16 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1] 21 55
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 56
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] 13 69
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 70
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 71
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] 13 84
Building 8 [1, 1, 1, 1, 1, 1, 0, 0] 7 91
Building 4 [0, 0, 1, 1] 5 96
Building 2 [1, 1] 1 97
Building 4 [1, 1, 1, 1] 1 98
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 99
Building

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5413
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5414
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5415
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5416
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5417
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5418
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5419
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5420
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5421
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5422
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5423
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5424
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5425
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5426
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Building 1 [0] 1 9569
Building 1 [0] 1 9570
Building 1 [0] 1 9571
Building 1 [0] 1 9572
Building 1 [0] 1 9573
Building 1 [0] 1 9574
Building 1 [0] 1 9575
Building 1 [0] 1 9576
Building 1 [0] 1 9577
Building 1 [0] 1 9578
Building 1 [0] 1 9579
Building 1 [0] 1 9580
Building 1 [0] 1 9581
Building 1 [0] 1 9582
Building 1 [0] 1 9583
Building 1 [0] 1 9584
Building 1 [0] 1 9585
Building 1 [0] 1 9586
Building 1 [0] 1 9587
Building 1 [0] 1 9588
Building 1 [0] 1 9589
Building 1 [0] 1 9590
Building 1 [0] 1 9591
Building 1 [0] 1 9592
Building 1 [0] 1 9593
Building 1 [0] 1 9594
Building 1 [0] 1 9595
Building 1 [0] 1 9596
Building 1 [0] 1 9597
Building 1 [0] 1 9598
Building 1 [0] 1 9599
Building 1 [0] 1 9600
Building 1 [0] 1 9601
Building 1 [0] 1 9602
Building 1 [0] 1 9603
Building 1 [0] 1 9604
Building 1 [0] 1 9605
Building 1 [0] 1 9606
Building 1 [0] 1 9607
Building 1 [0] 1 9608
Building 1 [0] 1 9609
Building 1 [0] 1 9610
Building 1 [0] 1 9611
Building 1 [0] 1 9612
Building 1 [0] 1 9613
Building 1

In [22]:
discard_list = []
for p in project_list:
    data = pd.read_csv('data/25_1_travis_data/'+p)
    
    branch_type = data['git_branch'].tolist()
    if 'master' in branch_type:
        data = data[ data['git_branch'] == 'master']
    else:
        data = data[ data['git_branch'] == 'trunk']
        
    data = data.iloc[100:]
    if len(data) == 0:
        print('Could not continue for {}'.format(p))
        discard_list.append(p)
        continue
        
    results = output_values(data['tr_status'].tolist())
    builds, delays = batchstop4_dynamic(results)
    reqd = 100-(100*builds/len(results))
    fr = 100*results.count(0)/len(results)
    
    p_name = p.split('/')[1]
    writer.writerow([p_name, 'BATCHSTOP4', '-1', reqd, median(delays), len(results)])
    
    #print('{:>40} {:>10} {:>30} {:>10}'.format(p, len(results), reqd, median(delays)))

Could not continue for BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv
Building 16 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1] 15 15
Building 8 [0, 1, 1, 1, 1, 1, 1, 1] 7 22
Building 4 [1, 1, 1, 1] 1 23
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 24
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 33
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 34
Building 16 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1] 19 53
Building 8 [1, 1, 1, 1, 0, 1, 1, 1] 7 60
Building 4 [0, 1, 1, 1] 5 65
Building 4 [0, 1, 1, 0] 5 70
Building 4 [1, 1, 0, 1] 5 75
Building 4 [1, 0, 1, 1] 5 80
Building 4 [1, 1, 1, 1] 1 81
Building 8 [1, 1, 1, 0, 1, 1, 1, 1] 7 88
Building 4 [1, 1, 1, 1] 1 89
Building 8 [1, 1, 1, 1, 1, 1, 0, 1] 7 96
Building 4 [0, 1, 1, 1] 5 101
Building 4 [1, 1, 0, 0] 5 106
Building 4 [1, 0, 1, 1] 5 111
Building 4 [0, 0, 0, 0] 5 116
Building 4 [1, 0, 1, 0] 5 121
Building 4 [0, 1, 1, 1] 5 126
Building 4 [1, 1, 1, 1] 1 127
Building 8 [0, 1, 1, 1, 1, 1, 1, 1] 7 134
Building 4 [0, 1, 1, 1] 5 13

Building 4 [1, 0, 0, 0] 5 330
Building 4 [0, 0, 1, 1] 5 335
Building 4 [1, 0, 0, 0] 5 340
Building 4 [0, 1, 1, 0] 5 345
Building 4 [0, 0, 0, 0] 5 350
Building 4 [0, 0, 0, 0] 5 355
Building 4 [0, 1, 1, 1] 5 360
Building 4 [1, 0, 0, 0] 5 365
Building 4 [0, 1, 1, 0] 5 370
Building 4 [1, 1, 0, 0] 5 375
Building 4 [0, 0, 0, 1] 5 380
Building 4 [1, 1, 0, 1] 5 385
Building 4 [1, 0, 0, 0] 5 390
Building 4 [0, 1, 1, 1] 5 395
Building 4 [1, 1, 1, 1] 1 396
Building 8 [1, 1, 0, 1, 1, 1, 0, 0] 11 407
Building 4 [0, 0, 0, 0] 5 412
Building 4 [1, 0, 0, 0] 5 417
Building 4 [0, 0, 0, 0] 5 422
Building 4 [0, 0, 0, 0] 5 427
Building 4 [0, 0, 0, 0] 5 432
Building 4 [0, 1, 1, 1] 5 437
Building 4 [1, 1, 0, 0] 5 442
Building 4 [0, 0, 1, 1] 5 447
Building 4 [0, 0, 0, 0] 5 452
Building 4 [0, 1, 1, 1] 5 457
Building 4 [1, 1, 1, 1] 1 458
Building 8 [0, 1, 1, 1, 0, 0, 0, 0] 13 471
Building 4 [0, 1, 1, 0] 5 476
Building 4 [0, 0, 0, 1] 5 481
Building 4 [1, 1, 1, 1] 1 482
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 483
Bu

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1] 9 9
Building 8 [1, 1, 1, 0, 1, 1, 1, 1] 7 16
Building 4 [1, 1, 1, 1] 1 17
Building 8 [0, 1, 1, 1, 1, 1, 1, 1] 7 24
Building 4 [1, 1, 1, 1] 1 25
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 26
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 27
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 28
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 29
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 30
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 31
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 32
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 33
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 34
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1] 9 43
Building 8 [1, 1, 1, 0, 1, 1, 1, 1] 7 50
Building 4 [1, 1, 1, 1] 1 51
Building 8 [1, 1, 1, 1, 0, 1, 1, 1] 7 58
Building 4 [1, 1, 1, 1] 1 59
Building 8 [0, 1, 1,

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 1
Building 16 [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1] 21 22
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 23
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 24
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 25
Building 16 [1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0] 19 44
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 45
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 46
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 47
Building 16 [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 13 60
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 61
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 62
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0] 9 71
Building 8 [0, 0, 1, 1, 1, 1, 1, 1] 7 78
Building 4 [1, 1, 1, 1] 1 79
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 80
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0] 11 91
Building 8 [0, 0, 0, 0, 0, 1, 1, 1] 1

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1] 9 9
Building 8 [1, 1, 1, 1, 0, 1, 1, 1] 7 16
Building 4 [1, 1, 1, 1] 1 17
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 18
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 19
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 20
Building 16 [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 31
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 32
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 33
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 34
Building 16 [1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 9 43
Building 8 [1, 1, 1, 0, 1, 1, 1, 1] 7 50
Building 4 [1, 1, 0, 1] 5 55
Building 4 [1, 1, 0, 1] 5 60
Building 4 [1, 1, 1, 1] 1 61
Building 8 [1, 1, 1, 1, 1, 1, 1, 0] 7 68
Building 4 [1, 1, 1, 1] 1 69
Building 8 [1, 0, 0, 1, 1, 1, 1, 1] 9 78
Building 4 [1, 1, 1, 1] 1 79
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 80
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 81
Building 16

...

Building 4 [0, 0, 0, 0] 5 7246
Building 4 [0, 0, 0, 0] 5 7251
Building 4 [0, 0, 0, 0] 5 7256
Building 4 [0, 0, 0, 0] 5 7261
Building 4 [0, 0, 0, 0] 5 7266
Building 4 [0, 0, 0, 0] 5 7271
Building 4 [0, 0, 0, 0] 5 7276
Building 4 [0, 0, 0, 0] 5 7281
Building 4 [0, 0, 0, 0] 5 7286
Building 4 [0, 0, 0, 0] 5 7291
Building 4 [0, 0, 0, 0] 5 7296
Building 4 [0, 0, 0, 0] 5 7301
Building 4 [0, 0, 0, 0] 5 7306
Building 4 [0, 0, 0, 0] 5 7311
Building 4 [0, 0, 0, 0] 5 7316
Building 4 [0, 0, 0, 0] 5 7321
Building 4 [0, 0, 0, 0] 5 7326
Building 4 [0, 0, 0, 0] 5 7331
Building 4 [0, 0, 0, 0] 5 7336
Building 4 [0, 0, 0, 0] 5 7341
Building 4 [0, 0, 0, 0] 5 7346
Building 4 [0, 0, 0, 0] 5 7351
Building 4 [0, 0, 0, 0] 5 7356
Building 4 [0, 0, 0, 0] 5 7361
Building 4 [0, 0, 0, 0] 5 7366
Building 4 [0, 0, 0, 0] 5 7371
Building 4 [0, 0, 0, 0] 5 7376
Building 4 [0, 0, 0, 0] 5 7381
Building 4 [0, 0, 0, 0] 5 7386
Building 4 [0, 0, 0, 0] 5 7391
Building 4 [0, 0, 0, 0] 5 7396
Building 4 [0, 0, 0, 0] 5 7401
Building

Building 4 [0, 0, 0, 0] 5 17277
Building 4 [0, 0, 0, 0] 5 17282
Building 4 [0, 0, 0, 0] 5 17287
Building 4 [0, 0, 0, 0] 5 17292
Building 4 [0, 0, 0, 0] 5 17297
Building 4 [0, 0, 0, 0] 5 17302
Building 4 [0, 0, 0, 0] 5 17307
Building 4 [0, 0, 0, 0] 5 17312
Building 4 [0, 0, 0, 0] 5 17317
Building 4 [0, 0, 0, 0] 5 17322
Building 4 [0, 0, 0, 0] 5 17327
Building 4 [0, 0, 0, 0] 5 17332
Building 4 [0, 0, 0, 0] 5 17337
Building 4 [0, 0, 0, 0] 5 17342
Building 4 [0, 0, 0, 0] 5 17347
Building 4 [0, 0, 0, 0] 5 17352
Building 4 [0, 0, 0, 0] 5 17357
Building 4 [0, 0, 0, 0] 5 17362
Building 4 [0, 0, 0, 0] 5 17367
Building 4 [0, 0, 0, 0] 5 17372
Building 4 [0, 0, 0, 0] 5 17377
Building 4 [0, 0, 0, 0] 5 17382
Building 4 [0, 0, 0, 0] 5 17387
Building 4 [0, 0, 0, 0] 5 17392
Building 4 [0, 0, 0, 0] 5 17397
Building 4 [0, 0, 0, 0] 5 17402
Building 4 [0, 0, 0, 0] 5 17407
Building 4 [0, 0, 0, 0] 5 17412
Building 4 [0, 0, 0, 0] 5 17417
Building 4 [0, 0, 0, 0] 5 17422
Building 4 [0, 0, 0, 0] 5 17427
Building

Building 16 [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 13 13
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 14
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 15
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 16
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 17
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1] 11 28
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 29
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 30
Building 16 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1] 23 53
Building 8 [0, 0, 0, 1, 1, 1, 1, 1] 9 62
Building 4 [1, 1, 1, 1] 1 63
Building 8 [1, 1, 1, 0, 0, 0, 1, 1] 11 74
Building 4 [1, 0, 0, 0] 5 79
Building 4 [0, 0, 0, 0] 5 84
Building 4 [0, 0, 0, 0] 5 89
Building 4 [0, 0, 0, 0] 5 94
Building 4 [1, 1, 1, 1] 1 95
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 96
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 105
Building 8 [0, 0, 0, 0, 0, 0, 0, 0] 15 120
Building 4 [0, 0, 0, 0] 5 125
Building 4 [0, 

Building 16 [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 13 13
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 14
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 15
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 16
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 17
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 18
Building 16 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1] 15 33
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 34
Building 16 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1] 21 55
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 56
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] 13 69
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 70
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 71
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] 13 84
Building 8 [1, 1, 1, 1, 1, 1, 0, 0] 7 91
Building 4 [0, 0, 1, 1] 5 96
Building 4 [1, 1, 1, 1] 1 97
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 98
Building 16 [1, 1, 0, 0, 0, 0, 

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8427
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8428
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8429
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8430
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8431
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8432
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8433
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8434
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8435
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8436
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8437
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8438
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8439
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8440
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,